In [ ]:
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.svm import SVR
import pandas as pd

# Carregar o dataset para treino
dataset = pd.read_csv('dataset.csv')

# Definir colunas categóricas
categorical_columns = ['genero', 'estado_civil', 'zona_residencia', 'fumador', 'class_etaria']

# One-hot encoding para colunas categóricas
encoder = OneHotEncoder(sparse_output=False)
one_hot_encoded = encoder.fit_transform(dataset[categorical_columns])
one_hot_df = pd.DataFrame(one_hot_encoded, columns=encoder.get_feature_names_out(categorical_columns))
dataset_encoded = pd.concat([dataset.drop(categorical_columns, axis=1), one_hot_df], axis=1)

threshold = 80000

outliers = dataset_encoded[(dataset_encoded['custo'] > threshold) | (dataset_encoded['imc'] < 10)]

dataset_encoded = dataset_encoded[~dataset_encoded.index.isin(outliers.index)]

# Separar variáveis independentes e dependente
X = dataset_encoded.drop(columns=['custo'])
y = dataset_encoded['custo']

# Normalizar os dados independentes
scaler_X = StandardScaler()
X_scaled = scaler_X.fit_transform(X)

# Instanciar e treinar o modelo SVM
svm_model = SVR(C=10, degree=3, epsilon=0.1, kernel='poly', gamma=1)
svm_model.fit(X_scaled, y)

# Carregar o dataset com novas features para predição
dataset_features = pd.read_csv('just_features.csv')

# One-hot encoding para o dataset de predição
one_hot_encoded_features = encoder.transform(dataset_features[categorical_columns])
one_hot_df_features = pd.DataFrame(one_hot_encoded_features, columns=encoder.get_feature_names_out(categorical_columns))
dataset_features_encoded = pd.concat([dataset_features.drop(categorical_columns, axis=1), one_hot_df_features], axis=1)

# Normalizar as features de entrada
X_scaled_features = scaler_X.transform(dataset_features_encoded)

# Predições do modelo
y_pred = svm_model.predict(X_scaled_features)

dataset_features['custo'] = y_pred
dataset_features['custo'].to_csv('grupo25custos_estimados.csv', index=False)